### 卷积网络分类 MNIST 数据集

**数据准备**
+ 读取训练和测试数据集
+ 图像数据转换为$28\times28\times1$的张量
+ 像素值变换到$[0,1]$
+ 类别标记变换为one-hot编码

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

train_data = pd.read_csv("MNIST_train.csv")
train_images = train_data.iloc[:,1:785].to_numpy()
train_images = train_images.reshape(60000,28,28,1)/255

train_labels = train_data.iloc[:,0].to_numpy()
train_labels = to_categorical(train_labels)

print("Shape of train samples:", train_images.shape)
print("Shape of train labels:", train_labels.shape)

test_data = pd.read_csv("MNIST_test.csv")
test_images = test_data.iloc[:,1:785].to_numpy()
test_images = test_images.reshape(10000,28,28,1)/255

test_labels = test_data.iloc[:,0].to_numpy()
test_labels = to_categorical(test_labels)

print("\nShape of test samples:", test_images.shape)
print("Shape of test labelss:", test_labels.shape)

Shape of train samples: (60000, 28, 28, 1)
Shape of train labels: (60000, 10)

Shape of test samples: (10000, 28, 28, 1)
Shape of test labelss: (10000, 10)


**构建网络**
+ 输入层：$28\times 28 \times 1$ 张量
+ 卷积层1：$32$个$3\times 3$的卷积核
+ 池化层1：最大值池化
+ 输出层2：$64$个$3\times 3$的卷积核
+ 池化层2：最大值池化
+ 输出层3：$64$个$3\times 3$的卷积核
+ 全连接层：$64$个神经元
+ 输出层：$10$个神经元，Softmax激活函数

In [2]:
from tensorflow.keras import models
from tensorflow.keras import layers

network = models.Sequential()
network.add(layers.Input(shape=(28,28,1)))
network.add(layers.Conv2D(32,(3,3),activation='relu'))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Conv2D(64,(3,3),activation='relu'))
network.add(layers.MaxPooling2D((2, 2)))
network.add(layers.Conv2D(64,(3,3),activation='relu'))

network.add(layers.Flatten())
network.add(layers.Dense(64, activation='relu'))
network.add(layers.Dense(10, activation='softmax'))

network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 576)               0

**网络学习和测试**
+ 优化算法：rmsprop
+ 损失函数：交叉熵
+ 以分类准确率作为度量
+ 学习5个回合
+ 每个回合迭代938个batch，每个batch 64个训练样本
+ 评估测试集的分类正确率

In [3]:
network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
network.fit(train_images, train_labels, epochs=5, batch_size=64)
print('\n')

test_loss, test_acc = network.evaluate(test_images, test_labels)
print('Test Accuracy:', test_acc)

Epoch 1/5
938/938 [==============================] - 19s 5ms/step - loss: 0.1637 - accuracy: 0.9495
Epoch 2/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0476 - accuracy: 0.9857
Epoch 3/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0328 - accuracy: 0.9900
Epoch 4/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0244 - accuracy: 0.9924
Epoch 5/5
938/938 [==============================] - 5s 5ms/step - loss: 0.0196 - accuracy: 0.9938


313/313 [==============================] - 1s 3ms/step - loss: 0.0432 - accuracy: 0.9887
Test Accuracy: 0.9886999726295471
